In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 加载乳腺癌数据集
cancer = load_breast_cancer()
X_cancer = cancer.data
y_cancer = cancer.target

In [21]:
# 数据预处理
scaler = StandardScaler()
X_cancer = scaler.fit_transform(X_cancer)

# 转换为 PyTorch 张量
X_tensor = torch.tensor(X_cancer, dtype=torch.float32)
y_tensor = torch.tensor(y_cancer, dtype=torch.float32)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=0)

In [22]:
# 定义包含可调整k的线性模型
class LinearModelWithTanhK(nn.Module):
    def __init__(self):
        super(LinearModelWithTanhK, self).__init__()
        self.linear = nn.Linear(X_cancer.shape[1], 1)
        self.k = nn.Parameter(torch.tensor([1.0]))  # 初始化k为可学习的参数

    def forward(self, x):
        x = self.linear(x)
        return torch.tanh(self.k * x)

# 优化函数
def optimize_and_update_k(model, optimizer, X, y, method="GD", batch_size=10, update_interval=5, increment=1.0, max_k=50.0):
    global epoch
    total_loss = 0.0

    if method == "SGD":
        indices = torch.randperm(len(X))
        for i in range(len(X)):
            optimizer.zero_grad()
            output = model(X[indices[i]:indices[i]+1]).squeeze()
            loss = F.binary_cross_entropy_with_logits(output.unsqueeze(0), y[indices[i]:indices[i]+1])
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
    elif method == "Mini-batch SGD":
        indices = torch.randperm(len(X))
        for i in range(0, len(X), batch_size):
            optimizer.zero_grad()
            output = model(X[indices[i:i+batch_size]]).squeeze()
            loss = F.binary_cross_entropy_with_logits(output, y[indices[i:i+batch_size]])
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
    else:  # GD
        optimizer.zero_grad()
        output = model(X).squeeze()
        loss = F.binary_cross_entropy_with_logits(output, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # 更新k的值
    if epoch % update_interval == 0 and model.k.item() < max_k:
        with torch.no_grad():
            model.k.add_(increment)

    return total_loss / len(X)

In [23]:
# 评估函数
def evaluate(model, X, y):
    with torch.no_grad():
        outputs = model(X).squeeze()
        predicted = torch.sigmoid(outputs) >= 0.5
        accuracy = (predicted == y).float().mean().item()
    return accuracy

In [24]:
# 初始化模型和优化器
lr = 0.01
num_epochs = 300
increment = 0.5


# 梯度下降（GD）
model_gd = LinearModelWithTanhK()
optimizer_gd = torch.optim.SGD(model_gd.parameters(), lr=lr)
for epoch in range(num_epochs):
    optimize_and_update_k(model_gd, optimizer_gd, X_train, y_train, method="GD", update_interval=5, increment=increment)
    if epoch % 20 == 0:
        acc_gd = evaluate(model_gd, X_test, y_test)
        print(f"GD - Epoch: {epoch}, Accuracy: {acc_gd:.4f}, k: {model_gd.k.item():.4f}")
print(f"Final Epoch {epoch}, Accuracy = {acc_gd:.4f}, k = {model_gd.k.item()}\n")

# 随机梯度下降（SGD）
model_sgd = LinearModelWithTanhK()
optimizer_sgd = torch.optim.SGD(model_sgd.parameters(), lr=lr)
for epoch in range(num_epochs):
    optimize_and_update_k(model_sgd, optimizer_sgd, X_train, y_train, method="SGD", update_interval=5, increment=increment)
    if epoch % 20== 0:
        acc_sgd = evaluate(model_sgd, X_test, y_test)
        print(f"SGD - Epoch: {epoch}, Accuracy: {acc_sgd:.4f}, k: {model_sgd.k.item():.4f}")
print(f"Final Epoch {epoch}, Accuracy = {acc_sgd:.4f}, k = {model_sgd.k.item()}\n")

# 小批量梯度下降（Mini-batch SGD）
model_mini_batch_sgd = LinearModelWithTanhK()
optimizer_mini_batch_sgd = torch.optim.SGD(model_mini_batch_sgd.parameters(), lr=lr)
for epoch in range(num_epochs):
    optimize_and_update_k(model_mini_batch_sgd, optimizer_mini_batch_sgd, X_train, y_train, method="Mini-batch SGD", update_interval=5, increment=increment)
    if epoch % 20 == 0:
        acc_mini_batch_sgd = evaluate(model_mini_batch_sgd, X_test, y_test)
        print(f"Mini-batch SGD - Epoch: {epoch}, Accuracy: {acc_mini_batch_sgd:.4f}, k: {model_mini_batch_sgd.k.item():.4f}")
print(f"Final Epoch {epoch}, Accuracy = {acc_mini_batch_sgd:.4f},  k = {model_mini_batch_sgd.k.item()}")

GD - Epoch: 0, Accuracy: 0.4386, k: 1.4996
GD - Epoch: 20, Accuracy: 0.8509, k: 3.5004
GD - Epoch: 40, Accuracy: 0.9123, k: 5.5011
GD - Epoch: 60, Accuracy: 0.9298, k: 7.5013
GD - Epoch: 80, Accuracy: 0.9298, k: 9.5013
GD - Epoch: 100, Accuracy: 0.9298, k: 11.5013
GD - Epoch: 120, Accuracy: 0.9298, k: 13.5014
GD - Epoch: 140, Accuracy: 0.9386, k: 15.5014
GD - Epoch: 160, Accuracy: 0.9386, k: 17.5014
GD - Epoch: 180, Accuracy: 0.9474, k: 19.5015
GD - Epoch: 200, Accuracy: 0.9474, k: 21.5015
GD - Epoch: 220, Accuracy: 0.9474, k: 23.5015
GD - Epoch: 240, Accuracy: 0.9474, k: 25.5015
GD - Epoch: 260, Accuracy: 0.9474, k: 27.5015
GD - Epoch: 280, Accuracy: 0.9474, k: 29.5015
Final Epoch 299, Accuracy = 0.9474, k = 31.00152587890625

SGD - Epoch: 0, Accuracy: 0.9474, k: 1.7229
SGD - Epoch: 20, Accuracy: 0.9649, k: 4.0529
SGD - Epoch: 40, Accuracy: 0.9561, k: 6.1160
SGD - Epoch: 60, Accuracy: 0.9561, k: 8.1299
SGD - Epoch: 80, Accuracy: 0.9561, k: 10.1332
SGD - Epoch: 100, Accuracy: 0.9561, k

In [25]:
class Perceptron:
    def __init__(self, num_features):
        self.weights = torch.zeros(num_features, dtype=torch.float32)
        self.bias = torch.tensor([0.0], dtype=torch.float32)

    def forward(self, x):
        linear_output = torch.dot(x, self.weights) + self.bias
        return torch.sign(linear_output)

    def update_weights(self, x, y, lr=0.01):
        prediction = self.forward(x)
        self.weights += lr * (y - prediction) * x
        self.bias += lr * (y - prediction)

# 评估函数
def evaluate(model, X, y):
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y_true in zip(X, y):
            prediction = model.forward(x)
            if prediction == y_true:
                correct += 1
            total += 1
    return correct / total

perceptron = Perceptron(num_features=X_train.shape[1])

for epoch in range(num_epochs):
    for x, y in zip(X_train, y_train):
        perceptron.update_weights(x, y, lr=0.01)

    if epoch % 20 == 0:
        accuracy = evaluate(perceptron, X_test, y_test)
        print(f'Epoch: {epoch}, Accuracy: {accuracy:.4f}')

Epoch: 0, Accuracy: 0.5614
Epoch: 20, Accuracy: 0.5789
Epoch: 40, Accuracy: 0.5614
Epoch: 60, Accuracy: 0.5351
Epoch: 80, Accuracy: 0.5789
Epoch: 100, Accuracy: 0.5789
Epoch: 120, Accuracy: 0.5702
Epoch: 140, Accuracy: 0.5702
Epoch: 160, Accuracy: 0.5175
Epoch: 180, Accuracy: 0.5702
Epoch: 200, Accuracy: 0.5702
Epoch: 220, Accuracy: 0.5702
Epoch: 240, Accuracy: 0.5702
Epoch: 260, Accuracy: 0.5702
Epoch: 280, Accuracy: 0.5702


In [26]:
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score

# 创建感知机模型
clf = Perceptron()

# 训练模型
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Epoch: {epoch}, Accuracy: {accuracy:.4f}')

Epoch: 299, Accuracy: 0.9298
